In [1]:
import os
import boto3
import pandas as pd

from io import BytesIO
from dotenv import load_dotenv

load_dotenv()

ACCESS_KEY = os.getenv('AK')
SECRET_KEY = os.getenv('SK')

In [17]:
from country_list import countries_for_language

In [21]:
countries = [val[1] for val in countries_for_language('en')]

In [22]:
countries

['Afghanistan',
 'Åland Islands',
 'Albania',
 'Algeria',
 'American Samoa',
 'Andorra',
 'Angola',
 'Anguilla',
 'Antarctica',
 'Antigua & Barbuda',
 'Argentina',
 'Armenia',
 'Aruba',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahamas',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bermuda',
 'Bhutan',
 'Bolivia',
 'Bosnia & Herzegovina',
 'Botswana',
 'Bouvet Island',
 'Brazil',
 'British Indian Ocean Territory',
 'British Virgin Islands',
 'Brunei',
 'Bulgaria',
 'Burkina Faso',
 'Burundi',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Cape Verde',
 'Caribbean Netherlands',
 'Cayman Islands',
 'Central African Republic',
 'Chad',
 'Chile',
 'China',
 'Christmas Island',
 'Cocos (Keeling) Islands',
 'Colombia',
 'Comoros',
 'Congo - Brazzaville',
 'Congo - Kinshasa',
 'Cook Islands',
 'Costa Rica',
 'Côte d’Ivoire',
 'Croatia',
 'Cuba',
 'Curaçao',
 'Cyprus',
 'Czechia',
 'Denmark',
 'Djibouti',
 'Dominica',
 'Dominican Republic',
 'Ecuador',
 'Egyp

In [2]:
s3_client = boto3.client('s3', aws_access_key_id=ACCESS_KEY, aws_secret_access_key=SECRET_KEY, region_name='us-east-1')

In [3]:
bucket_objs = s3_client.list_objects_v2(Bucket="unlabeled-news")

In [4]:
keys = [elem['Key'] for elem in bucket_objs['Contents']]

In [5]:
keys

['aevangelides/unlabeled_dataset.parquet.gzip',
 'amarinnunezdearce/unlabeled_dataset.parquet.gzip',
 'amontoya/unlabeled_dataset.parquet.gzip',
 'apillai/unlabeled_dataset.parquet.gzip',
 'ctoruno/unlabeled_dataset.parquet.gzip',
 'dbarba/unlabeled_dataset.parquet.gzip',
 'dhabiby/unlabeled_dataset.parquet.gzip',
 'ecampbell/unlabeled_dataset.parquet.gzip',
 'gdurbano/unlabeled_dataset.parquet.gzip',
 'ghulseman/unlabeled_dataset.parquet.gzip',
 'gnunezperalta/unlabeled_dataset.parquet.gzip',
 'hortiz/unlabeled_dataset.parquet.gzip',
 'hrigazzi/unlabeled_dataset.parquet.gzip',
 'hsoukireyes/unlabeled_dataset.parquet.gzip',
 'jcullen/unlabeled_dataset.parquet.gzip',
 'jdavis/unlabeled_dataset.parquet.gzip',
 'kwanner/unlabeled_dataset.parquet.gzip',
 'lcleary/unlabeled_dataset.parquet.gzip',
 'llittlejohn/unlabeled_dataset.parquet.gzip',
 'lsolis/unlabeled_dataset.parquet.gzip',
 'mbasystiuk/unlabeled_dataset.parquet.gzip',
 'mrodriguez/unlabeled_dataset.parquet.gzip',
 'mwoodbury/unla

In [6]:
dfs = [pd.read_parquet(BytesIO(s3_client.get_object(Bucket="unlabeled-news", Key=elem['Key'])['Body'].read())) for elem in bucket_objs['Contents']]

In [ ]:
[len(df) for df in dfs]

In [ ]:
article_ids = [df.article_id.unique().tolist() for df in dfs]

In [ ]:
result = set(article_ids[0])
for s in article_ids[1:]:
    result.intersection_update(s)
print(result)

In [ ]:
len(result)